# JSON Schema Parser

Notes on the JSON schema / Traitlets package

**Goal**: write a function that, given a JSON Schema, will generate code for traitlets objects which provide equivalent validation.

## Links

- [JSON Schema Validation](http://json-schema.org/latest/json-schema-validation.html)
- [``jsonschema`` Python package](https://pypi.python.org/pypi/jsonschema)

## By-Hand Example

First confirm that we're doing things correctly with the ``jsonschema`` package:

In [1]:
import json
import jsonschema

simple_schema = {
    "type": "object",
    "properties": {
        "foo": {"type": "string"},
        "bar": {"type": "number"}
    }
}

In [2]:
good_instance = {
    "foo": "hello world",
    "bar": 3.141592653,
}
jsonschema.validate(good_instance, simple_schema)

In [3]:
bad_instance = {
    "foo" : 42,
    "bar" : "string"
}

jsonschema.validate(bad_instance, simple_schema)

ValidationError: 42 is not of type 'string'

Failed validating 'type' in schema['properties']['foo']:
    {'type': 'string'}

On instance['foo']:
    42

OK, now let's write a traitlets class that does the same thing:

In [4]:
import traitlets as T

class SimpleInstance(T.HasTraits):
    foo = T.Unicode()
    bar = T.Float()

In [5]:
SimpleInstance(**good_instance)

In [6]:
SimpleInstance(**bad_instance)

TraitError: The 'foo' trait of a SimpleInstance instance must be a unicode string, but a value of 42 <class 'int'> was specified.

## Roadmap

1. Start by recognizing all simple JSON types in the schema ("string", "number", "integer", "boolean", "null")

2. Next recognize compound simple types (i.e. list of valid types)

3. Next recognize arrays & enums

4. Next recognize objects & "$ref" definitions

5. Next recognize "anyOf", "oneOf", "allOf" definitions... first is essentially a traitlets Union, second is a Union where only one must match, and "allOf" is essentially a composite object (not sure if traitlets has that...)

6. Implement custom traitlets that support all the various validation keywords for each type.

7. Use [hypothesis](https://hypothesis.readthedocs.io) for testing?

### Differences

- JSONSchema ignores any keys/properties which are undefined. Traitlets warns, and in the future will raise an error for undefined keys/properties

### Interface

- all definitions should become their own ``T.HasTraits`` class
- Objects defined inline should also have their own class with a generated anonymous name
- Use Jinja templating; allow output to one file or multiple files with relative imports